Fonte: https://github.com/jackfrost1411/next_word_suggestor

In [ ]:
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re # para mexer com regexp
from keras.utils import to_categorical
from doc3 import training_doc3

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
print(training_doc3)


how many people live in atlanta georgia	Atlanta ( , stressed , locally ) is the capital of and the most populous city in the U.S. state of Georgia , with an estimated 2011 population of 432,427 .	1
how many people live in atlanta georgia	Atlanta is the cultural and economic center of the Atlanta metropolitan area , home to 5,457,831 people and the ninth largest metropolitan area in the United States .	1
how many people live in atlanta georgia	Atlanta is the county seat of Fulton County , and a small portion of the city extends eastward into DeKalb County .	0
how many people live in atlanta georgia	Atlanta was established in 1837 at the intersection of two railroad lines , and the city rose from the ashes of the Civil War to become a national center of commerce .	0
how many people live in atlanta georgia	In the decades following the Civil Rights Movement , during which the city earned a reputation as `` too busy to hate '' for the progressive views of its citizens and leaders , Atlanta

In [ ]:
cleaned = re.sub(r'\W+', ' ', training_doc3).lower()
tokens = word_tokenize(cleaned)

train_len = 4 # vamos usar 4 aqui e é esperado que a gente use 3 palavras na hora de fazer query
text_sequences = []
for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences), train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]


In [ ]:
k = np.random.choice(range(len(n_sequences)))
for i in sequences[k]:
  print(tokens[i])

metamorphosis
between
project
only


In [ ]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size) # one hot encoding
seq_len = train_inputs.shape[1]

print(train_inputs.shape)

(25642, 3)


In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
#model = load_model("mymodel.h5")

model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3, 3)              1365      
_________________________________________________________________
lstm (LSTM)                  (None, 3, 50)             10800     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 50)                2550      
_________________________________________________________________
dense_1 (Dense)              (None, 455)               23205     
Total params: 58,120
Trainable params: 58,120
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(train_inputs,train_targets,epochs=500,verbose=1)
model.save("mymodel.h5")

Epoch 1/500
44/44 [==============================] - 4s 7ms/step - loss: 6.1028 - accuracy: 0.0640
Epoch 2/500
44/44 [==============================] - 0s 8ms/step - loss: 5.4970 - accuracy: 0.0790
Epoch 3/500
44/44 [==============================] - 0s 7ms/step - loss: 5.0913 - accuracy: 0.0860
Epoch 4/500
44/44 [==============================] - 0s 7ms/step - loss: 4.9811 - accuracy: 0.0758
Epoch 5/500
44/44 [==============================] - 0s 7ms/step - loss: 4.8700 - accuracy: 0.0833
Epoch 6/500
44/44 [==============================] - 0s 7ms/step - loss: 4.8054 - accuracy: 0.0804
Epoch 7/500
44/44 [==============================] - 0s 7ms/step - loss: 4.7571 - accuracy: 0.0862
Epoch 8/500
44/44 [==============================] - 0s 7ms/step - loss: 4.7086 - accuracy: 0.0892
Epoch 9/500
44/44 [==============================] - 0s 7ms/step - loss: 4.7711 - accuracy: 0.0893
Epoch 10/500
44/44 [==============================] - 0s 7ms/step - loss: 4.6496 - accuracy: 0.0996
Epoch 11/

In [ ]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(encoded_text, pad_encoded)
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)

how many people
[6, 4, 3] [[6 4 3]]
Next word suggestion: visit
Next word suggestion: live
Next word suggestion: died


# Usando um texto do Kafka:

In [ ]:
from urllib.request import urlopen

data = urlopen('https://www.gutenberg.org/cache/epub/5200/pg5200.txt') 

In [ ]:
texto = (data.read()).decode('utf-8')

In [ ]:
print(texto)

﻿The Project Gutenberg EBook of Metamorphosis, by Franz Kafka
Translated by David Wyllie.

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org

** This is a COPYRIGHTED Project Gutenberg eBook, Details Below **
**     Please follow the copyright guidelines in this file.     **


Title: Metamorphosis

Author: Franz Kafka

Translator: David Wyllie

Release Date: August 16, 2005 [EBook #5200]
First posted: May 13, 2002
Last updated: May 20, 2012

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK METAMORPHOSIS ***




Copyright (C) 2002 David Wyllie.





  Metamorphosis
  Franz Kafka

Translated by David Wyllie



I


One morning, when Gregor Samsa woke from troubled dreams, he found
himself transformed in his bed into a horrible vermin.  He lay on
his armour-like back, and if he lif

In [ ]:
cleaned = re.sub(r'\W+', ' ', texto).lower()
tokens = word_tokenize(cleaned)

train_len = 3+1
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
        
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]

In [ ]:
vocabulary_size

3016

In [ ]:
len(n_sequences)

25642

In [ ]:
train_inputs = n_sequences[:, :-1]
train_targets = n_sequences[:, -1]
train_targets = to_categorical(train_targets, num_classes = vocabulary_size)
seq_len = train_inputs.shape[1]
print(train_inputs.shape)
print(train_targets[0])

(25642, 3)
[0. 0. 0. ... 0. 0. 0.]


In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
#model = load_model("mymodel.h5")

model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3, 3)              9048      
_________________________________________________________________
lstm_2 (LSTM)                (None, 3, 50)             10800     
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 3016)              153816    
Total params: 196,414
Trainable params: 196,414
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(train_inputs, train_targets, epochs=500, verbose=1)

Epoch 1/500
802/802 [==============================] - 8s 10ms/step - loss: 1.6693 - accuracy: 0.5638
Epoch 2/500
802/802 [==============================] - 8s 10ms/step - loss: 1.6762 - accuracy: 0.5620
Epoch 3/500
802/802 [==============================] - 8s 10ms/step - loss: 1.6775 - accuracy: 0.5603
Epoch 4/500
802/802 [==============================] - 8s 10ms/step - loss: 1.6629 - accuracy: 0.5629
Epoch 5/500
802/802 [==============================] - 8s 10ms/step - loss: 1.6616 - accuracy: 0.5654
Epoch 6/500
802/802 [==============================] - 8s 10ms/step - loss: 1.6631 - accuracy: 0.5620
Epoch 7/500
802/802 [==============================] - 8s 10ms/step - loss: 1.6546 - accuracy: 0.5659
Epoch 8/500
802/802 [==============================] - 8s 10ms/step - loss: 1.6596 - accuracy: 0.5654
Epoch 9/500
802/802 [==============================] - 8s 10ms/step - loss: 1.6685 - accuracy: 0.5619
Epoch 10/500
802/802 [==============================] - 8s 10ms/step - loss: 1.665

In [ ]:
model.save('lstfm_kafka1kepochs.h5') 
#model.load('lstfm_kafka1kepochs.h5')

In [ ]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

he was a
Next word suggestion: lieutenant
Next word suggestion: loud
Next word suggestion: end


In [ ]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

she was a
Next word suggestion: lover
Next word suggestion: lieutenant
Next word suggestion: child


In [ ]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

he had been
Next word suggestion: suddenly
Next word suggestion: reduced
Next word suggestion: drawn


In [ ]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

he has a
Next word suggestion: impression
Next word suggestion: chest
Next word suggestion: way


In [ ]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

she has a
Next word suggestion: chest
Next word suggestion: impression
Next word suggestion: way


In [ ]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

turn into the
Next word suggestion: headboard
Next word suggestion: home
Next word suggestion: left


In [ ]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

habit of
Next word suggestion: sweetened
Next word suggestion: anxiety
Next word suggestion: dirt


In [ ]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

he had the habit of
Next word suggestion: crawling
Next word suggestion: putting
Next word suggestion: closely


In [ ]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

now, he was in the
Next word suggestion: dark
Next word suggestion: lower
Next word suggestion: middle


In [ ]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

he liked his
Next word suggestion: father
Next word suggestion: sister
Next word suggestion: head


In [ ]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-5:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

please enjoy this
Next word suggestion: volunteers
Next word suggestion: means
Next word suggestion: page
Next word suggestion: copy
Next word suggestion: trademark
